In [113]:
import urllib.request

fp = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()

The following code parses HTML from the provided Wikipedia article. A number of assumptions are made, specifically that the table containing postal code information is the first table on the page and that the columns correspond to the expected values.

In [111]:
from bs4 import BeautifulSoup
postcodes = []
boroughs = []
neighborhoods = []

from bs4 import BeautifulSoup

soup = BeautifulSoup(mystr,"html.parser")
for table in soup.find_all('table'):
    for table_row in table.find_all('tr'):
        cells = table_row.find_all('td')
        cells = [ele.text.strip() for ele in cells]
        if (len(cells)>2):
            postcode = cells[0]
            borough = cells[1]
            neighborhood = cells[2]
            if (borough != 'Not assigned'):
                if (neighborhood == 'Not assigned'):
                    neighborhood = borough
                if (postcode in postcodes):
                    postcode_index = postcodes.index(postcode)
                    neighborhoods[postcode_index] = neighborhoods[postcode_index] + ", " + neighborhood
                else:
                    postcodes.append(postcode);
                    boroughs.append(borough);
                    neighborhoods.append(neighborhood);
    break
    


In [112]:
import pandas as pd
df = pd.DataFrame({'Postal Code':postcodes,'Borough':boroughs,'Neighborhood':neighborhoods})
df

,Borough,Neighborhood,Postal Code
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
3,North York,"Lawrence Heights, Lawrence Manor",M6A
4,Queen's Park,Queen's Park,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Rouge, Malvern",M1B
7,North York,Don Mills North,M3B
8,East York,"Woodbine Gardens, Parkview Hill",M4B
9,Downtown Toronto,"Ryerson, Garden District",M5B


In [109]:
print(df.shape)

(103, 3)
